In [49]:
import pandas as pd
import requests
import numpy as np
pd.set_option('display.max_columns', None)

In [24]:
header = {"User-Agent": "Chrome/84.0.4147.89","X-Requested-With": "XMLHttpRequest"}

In [25]:
url = 'https://fundamentus.com.br/resultado.php'

In [26]:
empresas_listadas = pd.read_html(requests.get(url, headers = header).text,decimal=',',thousands='.')[0]

In [27]:
empresas_listadas.head()

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,PORP4,2.40,0.0,0.0,0.0,"0,00%",0.0,0.0,0.0,0.0,0.0,0.0,"0,00%","0,00%",0.00,"0,00%","-2,08%",0.0,2.239900e+07,0.00,"13,66%"
1,MNSA3,0.42,0.0,0.0,0.0,"0,00%",0.0,0.0,0.0,0.0,0.0,0.0,"-208,15%","-362,66%",3.63,"-13,50%","145,70%",0.0,-9.105000e+06,-6.52,"-41,11%"
2,CSTB3,150.00,0.0,0.0,0.0,"0,00%",0.0,0.0,0.0,0.0,0.0,0.0,"40,85%","28,98%",2.60,"22,40%","20,11%",0.0,8.420670e+09,0.14,"31,91%"
3,CFLU4,1000.00,0.0,0.0,0.0,"0,00%",0.0,0.0,0.0,0.0,0.0,0.0,"8,88%","10,72%",1.10,"17,68%","32,15%",0.0,6.035100e+07,0.06,"8,14%"
4,POPR4,10.17,0.0,0.0,0.0,"0,00%",0.0,0.0,0.0,0.0,0.0,0.0,"8,66%","5,65%",1.08,"15,25%","19,93%",0.0,5.458030e+08,0.82,"30,93%"


In [28]:
colunas = ['Div.Yield','Mrg Ebit', 'Mrg. Líq.','ROIC', 'ROE','Cresc. Rec.5a']
for i in colunas:
    empresas_listadas[i] = empresas_listadas[i].str.rstrip('%')
    empresas_listadas[i] = empresas_listadas[i].str.replace('.','').str.replace(',','.').astype(float)
empresas_listadas.head()

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,PORP4,2.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,-2.08,0.0,2.239900e+07,0.00,13.66
1,MNSA3,0.42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-208.15,-362.66,3.63,-13.50,145.70,0.0,-9.105000e+06,-6.52,-41.11
2,CSTB3,150.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.85,28.98,2.60,22.40,20.11,0.0,8.420670e+09,0.14,31.91
3,CFLU4,1000.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.88,10.72,1.10,17.68,32.15,0.0,6.035100e+07,0.06,8.14
4,POPR4,10.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.66,5.65,1.08,15.25,19.93,0.0,5.458030e+08,0.82,30.93


In [29]:
empresas_listadas.columns

Index(['Papel', 'Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo',
       'P/Cap.Giro', 'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA',
       'Mrg Ebit', 'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses',
       'Patrim. Líq', 'Dív.Brut/ Patrim.', 'Cresc. Rec.5a'],
      dtype='object')

In [30]:
# Primeiramente, filtramos empresas com liquidez acima de 500k

print(empresas_listadas.shape)
magic_formula = empresas_listadas.loc[empresas_listadas['Liq.2meses'] >= 500000]
print(magic_formula.shape)

(957, 21)
(300, 21)


In [45]:
# Agora, filtramos somente as empresas com resultado liquido positivo

magic_formula = magic_formula.loc[(magic_formula['ROIC'] > 0) &
                                 (magic_formula['P/L'] > 0) &
                                 (magic_formula['ROE'] > 0)&
                                  (magic_formula['EV/EBIT'] > 0)&
                                 (magic_formula['Mrg. Líq.'] > 0)&
                                 (magic_formula['Div.Yield'] < 20)].reset_index(drop=True)
print(magic_formula.shape)

(183, 21)


In [46]:
magic_formula

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,ATOM3,7.32,11.83,9.87,4.201,4.10,4.782,6.43,12.03,10.25,10.90,10.84,34.92,35.50,4.12,73.03,83.41,1465120.0,1.550100e+07,0.00,85.17
1,WIZS3,12.64,9.77,5.84,2.357,4.82,2.112,47.97,4.80,-9.17,4.18,3.68,49.09,24.73,1.13,67.18,59.77,45698500.0,3.461390e+08,0.00,17.59
2,CMIN3,9.71,9.06,4.10,3.084,5.31,2.490,12.58,5.61,351.84,5.23,4.63,54.96,34.03,1.98,64.23,45.24,114909000.0,1.324460e+10,0.11,0.00
3,PSSA3,54.25,9.99,2.02,0.925,5.16,0.478,5.36,1.17,-5.18,0.52,0.51,79.38,9.26,1.15,55.89,20.23,53596500.0,8.682600e+09,0.00,3.80
4,ODPV3,12.81,17.25,5.25,3.863,5.96,3.349,12.36,10.90,18.24,9.54,9.17,35.45,22.37,1.99,53.73,30.46,13426800.0,1.295490e+09,0.00,7.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,BRPR3,9.53,22.81,0.64,14.641,2.07,0.423,7.81,21.09,-1.69,29.27,29.15,69.42,64.18,2.44,2.17,2.82,13822000.0,7.285270e+09,0.36,-8.30
179,HGTX3,34.50,15.68,3.41,5.161,1.59,2.517,6.99,202.43,7.79,195.24,46.99,2.55,32.92,2.60,1.75,21.76,129558000.0,1.643810e+09,0.10,-6.92
180,ITSA4,11.73,11.96,1.68,15.211,2.53,1.346,24.03,116.89,-23.94,118.85,66.56,13.01,132.83,2.28,1.24,14.06,333877000.0,5.869600e+10,0.10,6.50
181,ITSA3,12.07,12.30,1.73,15.652,2.46,1.385,24.72,120.28,-24.63,122.24,68.46,13.01,132.83,2.28,1.24,14.06,2312010.0,5.869600e+10,0.10,6.50


In [52]:
# Ordenar do maior ROIC para o menor

magic_formula.sort_values('ROIC', ascending = False, inplace = True)
magic_formula['Ordena_ROIC'] = np.arange(magic_formula.shape[0])
magic_formula.head()

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Ordena_ROIC
0,ATOM3,7.32,11.83,9.87,4.201,4.10,4.782,6.43,12.03,10.25,10.90,10.84,34.92,35.50,4.12,73.03,83.41,1465120.0,1.550100e+07,0.00,85.17,0
1,WIZS3,12.64,9.77,5.84,2.357,4.82,2.112,47.97,4.80,-9.17,4.18,3.68,49.09,24.73,1.13,67.18,59.77,45698500.0,3.461390e+08,0.00,17.59,1
2,CMIN3,9.71,9.06,4.10,3.084,5.31,2.490,12.58,5.61,351.84,5.23,4.63,54.96,34.03,1.98,64.23,45.24,114909000.0,1.324460e+10,0.11,0.00,2
3,PSSA3,54.25,9.99,2.02,0.925,5.16,0.478,5.36,1.17,-5.18,0.52,0.51,79.38,9.26,1.15,55.89,20.23,53596500.0,8.682600e+09,0.00,3.80,3
4,ODPV3,12.81,17.25,5.25,3.863,5.96,3.349,12.36,10.90,18.24,9.54,9.17,35.45,22.37,1.99,53.73,30.46,13426800.0,1.295490e+09,0.00,7.21,4


In [53]:
# Ordenar o EV/EBIT do menor para o maior

magic_formula.sort_values('EV/EBIT', ascending = True, inplace = True)
magic_formula['Ordena_EV/EBIT'] = np.arange(magic_formula.shape[0])
magic_formula.head()

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Ordena_ROIC,Ordena_EV/EBIT
136,GPIV33,5.37,7.43,0.39,2.181,0.00,0.177,0.93,3.27,25.14,0.35,0.35,66.73,49.68,3.91,6.93,5.31,1570860.0,1.493730e+09,0.12,-12.57,136,0
3,PSSA3,54.25,9.99,2.02,0.925,5.16,0.478,5.36,1.17,-5.18,0.52,0.51,79.38,9.26,1.15,55.89,20.23,53596500.0,8.682600e+09,0.00,3.80,3,1
66,GOAU3,13.80,9.40,1.17,0.295,3.26,0.216,0.91,2.08,-2.71,3.35,2.49,14.19,9.10,2.44,13.30,12.41,1397370.0,1.285910e+10,1.38,7.17,66,2
67,GOAU4,14.79,10.08,1.25,0.316,3.04,0.231,0.98,2.22,-2.91,3.50,2.60,14.19,9.10,2.44,13.30,12.41,157997000.0,1.285910e+10,1.38,7.17,67,3
15,MRFG3,18.61,3.56,5.70,0.186,1.10,0.317,2.95,1.54,-0.70,3.68,3.14,12.11,7.12,1.31,30.68,160.09,196653000.0,2.322490e+09,12.71,46.05,15,4


In [57]:
# SUMARIZAÇÃO

magic_formula['VALOR_FORMULA'] = magic_formula['Ordena_EV/EBIT'] + magic_formula['Ordena_ROIC']
magic_formula.sort_values('VALOR_FORMULA', ascending = True, inplace = True)
print("Carteira da MAGIC FORMULA:")
magic_formula.head(20)

Carteira da MAGIC FORMULA:


,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Ordena_ROIC,Ordena_EV/EBIT,VALOR_FORMULA
3,PSSA3,54.25,9.99,2.02,0.925,5.16,0.478,5.36,1.17,-5.18,0.52,0.51,79.38,9.26,1.15,55.89,20.23,5.359650e+07,8.682600e+09,0.00,3.80,3,1,4
1,WIZS3,12.64,9.77,5.84,2.357,4.82,2.112,47.97,4.80,-9.17,4.18,3.68,49.09,24.73,1.13,67.18,59.77,4.569850e+07,3.461390e+08,0.00,17.59,1,8,9
2,CMIN3,9.71,9.06,4.10,3.084,5.31,2.490,12.58,5.61,351.84,5.23,4.63,54.96,34.03,1.98,64.23,45.24,1.149090e+08,1.324460e+10,0.11,0.00,2,16,18
15,MRFG3,18.61,3.56,5.70,0.186,1.10,0.317,2.95,1.54,-0.70,3.68,3.14,12.11,7.12,1.31,30.68,160.09,1.966530e+08,2.322490e+09,12.71,46.05,15,4,19
10,PLPL3,6.26,7.90,4.56,1.259,3.13,1.093,1.37,4.21,5.19,4.52,4.22,29.89,15.93,5.68,37.45,57.76,7.185250e+06,2.801500e+08,1.33,0.00,10,10,20
13,VALE3,114.34,10.73,2.91,2.450,5.83,1.219,9.35,4.35,-3.74,4.39,3.91,56.34,22.08,1.96,34.35,27.13,2.811470e+09,2.074870e+11,0.38,23.67,13,9,22
5,AURA33,63.81,8.36,2.95,2.271,7.18,1.395,12.34,5.72,-14.51,5.27,5.27,39.71,27.16,1.36,48.05,35.29,1.210760e+07,1.530100e+09,0.33,0.00,5,17,22
6,TASA3,23.95,4.91,15.64,1.184,0.00,1.413,5.81,4.33,-5.54,5.33,5.07,27.35,24.13,1.59,43.80,318.75,2.137870e+06,1.533560e+08,5.29,21.94,6,18,24
7,TASA4,24.18,4.95,15.79,1.196,0.00,1.427,5.87,4.37,-5.59,5.37,5.11,27.35,24.13,1.59,43.80,318.75,5.242580e+07,1.533560e+08,5.29,21.94,7,20,27
11,UNIP6,91.50,11.72,4.17,1.971,5.20,1.785,8.41,6.89,-15.21,6.50,5.60,28.63,17.01,1.89,36.26,35.55,4.388250e+07,2.074060e+09,0.27,25.71,12,32,44
